In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import merge
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import time
from datetime import datetime, timedelta

In [2]:
start = time.time()

In [3]:
#显示所有列
pd.set_option("display.max_columns", None) 
#显示所有行，慎用，容易死机
#pd.set_option("display.max_rows", None) 

# 1、读入数仓数据

In [4]:
# 读取year
def read_newyear():
    df = pd.read_csv('../JN_DataWarehouse/stock_analysis/TuShare/new_year.csv' ,index_col=False)
    
    del df['Unnamed: 0']
    return df

In [5]:
# 读取日历，按时间段
def read_cal_t(day):

    df = pd.read_excel('../JN_DataWarehouse/stock_analysis/TuShare/cal.xls', index_col=False)

    now = datetime.now().date()
    s = datetime.strftime(now, "%Y%m%d")
      
    df_cal_t = df.query("is_open == 1 and cal_date <= "+s+"")[['cal_date']].tail(day)  # query 查询语句

    return df_cal_t

In [6]:
df_stock_all = read_newyear()
df_stock_all.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,002007.SZ,20110104,48.51,48.69,47.42,48.13,48.39,-0.26,-0.54,50208.66,240624.7194
1,000088.SZ,20110104,6.15,6.18,6.09,6.17,6.13,0.04,0.65,22159.02,13623.5458
2,000068.SZ,20110104,6.01,6.30,5.94,6.30,6.00,0.30,5.00,109392.70,66681.5565
3,600167.SH,20110104,9.62,9.78,9.58,9.74,9.63,0.11,1.14,8403.49,8151.7650
4,002418.SZ,20110104,22.05,22.64,22.01,22.58,22.00,0.58,2.64,12609.19,28261.5130


# 2、 过滤6天数据

In [7]:
filter_day = 1000                # 待分析数据总天数
amount_day = 6                  # 每天分析数据周期去最近6天
df_cal_f = read_cal_t(filter_day)
df_cal_f.index=range(df_cal_f.shape[0])
df_cal_f.tail(100)

,cal_date
900,20200512
901,20200513
902,20200514
903,20200515
904,20200518
...,...
995,20200924
996,20200925
997,20200928
998,20200929


In [8]:
df3 = merge(df_stock_all,df_cal_f,left_on=["trade_date"],right_on=["cal_date"], how ='left' ) 
df3 = df3[df3['cal_date'].notnull()]
df3.pop("cal_date")
df3.info

<bound method DataFrame.info of            ts_code  trade_date   open   high    low  close  pre_close  change  \
6421443  603308.SH    20160824  21.36  21.53  21.16  21.41      21.38    0.03   
6421444  600691.SH    20160824   3.18   3.19   3.16   3.18       3.19   -0.01   
6421445  600785.SH    20160824  28.00  28.00  27.28  27.39      27.72   -0.33   
6421446  002301.SZ    20160824  21.14  21.24  20.90  21.12      21.12    0.00   
6421447  600033.SH    20160824   3.38   3.39   3.35   3.36       3.38   -0.02   
...            ...         ...    ...    ...    ...    ...        ...     ...   
9797497  603321.SH    20200928   7.49   7.59   7.36   7.37       7.49   -0.12   
9797498  603181.SH    20200928  17.35  17.50  16.96  17.04      17.25   -0.21   
9797499  300683.SZ    20200928  48.48  48.51  46.20  46.20      48.15   -1.95   
9797500  603557.SH    20200928  14.31  15.11  13.36  13.68      14.52   -0.84   
9797501  603183.SH    20200928   8.72   8.72   8.32   8.50       8.77   -0.27

In [9]:
# 查出上市未够6天的公司
df4 = df3.groupby(["ts_code"], as_index=False).count()
df5 = df4[df4.open<6]
df5

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
1441,002998.SZ,2,2,2,2,2,2,2,2,2,2
1444,003002.SZ,5,5,5,5,5,5,5,5,5,5
1446,003005.SZ,5,5,5,5,5,5,5,5,5,5
1448,003007.SZ,4,4,4,4,4,4,4,4,4,4
1449,003008.SZ,4,4,4,4,4,4,4,4,4,4
1450,003009.SZ,2,2,2,2,2,2,2,2,2,2
1451,003010.SZ,2,2,2,2,2,2,2,2,2,2
2319,300892.SZ,3,3,3,3,3,3,3,3,3,3
2320,300893.SZ,3,3,3,3,3,3,3,3,3,3
2321,300895.SZ,3,3,3,3,3,3,3,3,3,3


In [10]:
# 把df3里面ts_code 在df5里面的去掉，生成df_result
df_result = df3[~df3['ts_code'].isin(df5['ts_code'])]
#查看df5里面交易天数合计
#df5['trade_date'].sum()

In [11]:
# 重新排序，重新索引
df_result.index=range(df_result.shape[0])
df_result

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,603308.SH,20160824,21.36,21.53,21.16,21.41,21.38,0.03,0.1400,46676.90,99467.5980
1,600691.SH,20160824,3.18,3.19,3.16,3.18,3.19,-0.01,-0.3100,120034.33,38140.5870
2,600785.SH,20160824,28.00,28.00,27.28,27.39,27.72,-0.33,-1.1900,12202.91,33628.8760
3,002301.SZ,20160824,21.14,21.24,20.90,21.12,21.12,0.00,0.0000,37487.92,78860.4925
4,600033.SH,20160824,3.38,3.39,3.35,3.36,3.38,-0.02,-0.5900,116094.42,39152.4030
...,...,...,...,...,...,...,...,...,...,...,...
3375999,603321.SH,20200928,7.49,7.59,7.36,7.37,7.49,-0.12,-1.6021,22004.00,16362.4960
3376000,603181.SH,20200928,17.35,17.50,16.96,17.04,17.25,-0.21,-1.2174,18657.07,32062.9090
3376001,300683.SZ,20200928,48.48,48.51,46.20,46.20,48.15,-1.95,-4.0498,8406.13,39426.8580
3376002,603557.SH,20200928,14.31,15.11,13.36,13.68,14.52,-0.84,-5.7851,381646.73,543264.5310


In [12]:
def f_rsi_2( df_rsi):    
    
    # change 大于0
    df_tmp_1 = df_rsi.loc[~df_rsi.where(df_rsi['change']<0).any(axis=1)]

    # 绝对值change
    df_tmp_2 = df_rsi.copy()
    df_tmp_2['change'] = abs(df_tmp_2['change'])

    df_tmp_11 = df_tmp_1.groupby('ts_code')['change'].sum().to_frame('positive')

    df_tmp_21 = df_tmp_2.groupby('ts_code')['change'].sum().to_frame('p_sum')
    df_tmp_3 = df_tmp_21.join(df_tmp_11)
    df_tmp_3['rsi'] = (100-100/(1+df_tmp_3['positive']/df_tmp_3['p_sum'])).round(2)
    df_tmp_3.fillna(0,inplace=True)     # 把NaN转换为0，可以按列设置 
    df_tmp_3.pop('positive')
    df_tmp_3.pop('p_sum')
    
    return df_tmp_3

In [13]:
def f_rsi_1(df_cal_f, df_result, amount_day):
    df_cal_a = df_cal_f.copy()
    df_result_new = df_result.loc[df_result.where(df_result['ts_code']=='0').any(axis=1)]
    df_result_new['rsi']=0
  
    # 倒序遍历df_cal_f
    for index, row in df_cal_f.iloc[::-1].iterrows():
  

        if len(df_cal_a.index) > amount_day - 1 :       #抽取最近amount_day=6天数据进行分析处理，然后写入当天
        
            df_rsi = df_result[df_result["trade_date"].isin(df_cal_a['cal_date'].tail(amount_day))]
            
            df = df_result[df_result["trade_date"].isin(df_cal_a['cal_date'].tail(1))]
            df = df.set_index("ts_code", drop=False)
            #print( df_result[df_result.trade_date==row["cal_date"]] )
                       
            df_rsi_2 = f_rsi_2(df_rsi)
            
            df = df.join(df_rsi_2)
            
            df_result_new = df_result_new.append(df,ignore_index = True)
            
            if index == 35:
                print('df_cal_a count: ' + str(len(df_cal_a.index)))
                print('df_rsi   count: ' + str(len(df_rsi.index)))
                print('df_rsi_2 count: ' + str(len(df_rsi_2.index)))
                print('df       count: ' + str(len(df.index)))
                print(df_rsi.query("ts_code == '"+'000757.SZ'+"'")['trade_date'].tail(1))          
                       
           
            #df_result['rsi'][df_result["trade_date"].isin(df_cal_a['cal_date'].tail(1))]=df_result.apply(f_rsi_2, axis=1, args=('ts_code', df))
            
            df_cal_a = df_cal_a.drop(df_cal_a.tail(1).index)

    return df_result_new

In [14]:
df_rsi_1 = f_rsi_1(df_cal_f, df_result, amount_day)

df_cal_a count: 36
df_rsi   count: 16357
df_rsi_2 count: 2759
df       count: 2729
96729    20161021
Name: trade_date, dtype: int64


In [15]:
df_rsi_1[df_rsi_1.ts_code == '601398.SH'].head(200)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,rsi
1883,601398.SH,20200928,4.90,4.94,4.88,4.94,4.89,0.05,1.0225,1634647.01,803113.059,26.32
7823,601398.SH,20200925,4.90,4.91,4.88,4.89,4.90,-0.01,-0.2041,1080017.50,528791.477,23.53
10114,601398.SH,20200924,4.91,4.92,4.88,4.90,4.92,-0.02,-0.4065,1265965.12,619885.384,25.00
14216,601398.SH,20200923,4.93,4.94,4.91,4.92,4.93,-0.01,-0.2028,1005836.16,495260.688,33.33
16784,601398.SH,20200922,4.95,4.96,4.92,4.93,4.96,-0.03,-0.6048,1097050.20,542057.884,33.33
...,...,...,...,...,...,...,...,...,...,...,...,...
750014,601398.SH,20191211,5.71,5.75,5.69,5.75,5.70,0.05,0.8772,1128807.54,646313.686,27.27
751555,601398.SH,20191210,5.73,5.73,5.69,5.70,5.74,-0.04,-0.6969,977657.65,557745.079,7.14
756037,601398.SH,20191209,5.74,5.76,5.70,5.74,5.75,-0.01,-0.1739,1104982.01,632087.933,21.43
759554,601398.SH,20191206,5.77,5.79,5.72,5.75,5.77,-0.02,-0.3466,1096040.45,629666.588,16.67


In [16]:
print('finish:%.4f'%(time.time()-start))

finish:140.6198


In [17]:
dfb = pd.DataFrame({'a': ['one', 'one', 'two',
                           'three', 'two', 'one', 'six'],
                    'c': np.arange(7)})
 
##################################################################################################################################################################
# This will show the SettingWithCopyWarning
# but the frame values will be set
dfb['c'][dfb['a'].str.startswith('o')] = 42
dfb

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,a,c
0,one,42
1,one,42
2,two,2
3,three,3
4,two,4
5,one,42
6,six,6
